In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 59 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session

!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [ ]:
spark

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


In [ ]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

In [ ]:
doc_text_pairs = parquetFile.limit(200).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(200).select("title", "id").rdd
t0c_anchor_text = parquetFile.limit(200).select("anchor_text", "id").rdd

# doc_text_pairs.foreach(lambda x : print(x))



In [ ]:
print(t0c_anchor_text.take(2))

invered index


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)
import math

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  c = Counter(tokens)
  lst = [(item[0],(id,item[1])) for item in c.items() if item[0] not in all_stopwords]
  return lst


def lentext( text,id):
  lst = [(id,len(text))]
  return lst

def titledoc(title,id):
    lst = [(id,title)]
    return lst


def create_dominator(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    filtered_tokens = [tok for tok in tokens if tok not in all_stopwords]
    print(filtered_tokens)
    doc_len = len(filtered_tokens)
    wij = {} 
    sum = 0
    for token in filtered_tokens:
      if token in wij:
        wij[token] += 1
      else:
        wij[token] = 1
    
    for key in wij:
      sum +=math.pow(wij[key] / doc_len,2)
    return [(id,sum)]


      

In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_title_counts =  doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

total_idf = doc_text_pairs.flatMap(lambda x: create_dominator(x[0], x[1]))

title_per_id = doc_title_pairs.flatMap(lambda x: titledoc(x[0],x[1]))
len_text = doc_text_pairs.flatMap(lambda x : lentext(x[0],x[1]))
len_title = word_title_counts.flatMap(lambda x : lentext(x[0],x[1]))

In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key= lambda tup : tup[0])

In [ ]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)


Change it to 50 in the GCD

In [ ]:
postings_filtered = postings.filter(lambda x: len(x[1])>10)

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  def sumtf(arr):
      sum = 0 
      for tup in arr:
        sum = sum + len(tup)
      return sum

  rdd_sum =postings.groupByKey().mapValues(list)
  rdd = rdd_sum.map(lambda x : (x[0],sumtf(x[1])))

  return rdd




In [ ]:
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
id_to_title = title_per_id.collectAsMap()
DL = len_text.collectAsMap()
tfidf =  total_idf.collectAsMap()


In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  rdd_bk = postings.map(lambda x : (token2bucket_id(x[0]),(x[0],x[1])))
  rbb_groupbykk = rdd_bk.groupByKey().mapValues(list)
  rbb_wr = rbb_groupbykk.map(lambda x : InvertedIndex.write_a_posting_list(x))
  
  return rbb_wr

In [ ]:
posting_locs_list = partition_postings_and_write(postings_filtered).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

inverted.DL = DL 
inverted.N = len(DL)
inverted.id_to_title = id_to_title

inverted.tfidf_dom =tfidf
inverted.write_index('.', 'index')
# write the global stats out


In [ ]:
! zip posting.zip *.bin

  adding: 0_000.bin (deflated 67%)
  adding: 100_000.bin (deflated 65%)
  adding: 10_000.bin (deflated 64%)
  adding: 1_000.bin (deflated 64%)
  adding: 101_000.bin (deflated 65%)
  adding: 102_000.bin (deflated 65%)
  adding: 103_000.bin (deflated 64%)
  adding: 104_000.bin (deflated 65%)
  adding: 105_000.bin (deflated 66%)
  adding: 106_000.bin (deflated 65%)
  adding: 107_000.bin (deflated 63%)
  adding: 108_000.bin (deflated 65%)
  adding: 109_000.bin (deflated 65%)
  adding: 110_000.bin (deflated 66%)
  adding: 11_000.bin (deflated 66%)
  adding: 111_000.bin (deflated 66%)
  adding: 112_000.bin (deflated 67%)
  adding: 113_000.bin (deflated 66%)
  adding: 114_000.bin (deflated 64%)
  adding: 115_000.bin (deflated 64%)
  adding: 116_000.bin (deflated 66%)
  adding: 117_000.bin (deflated 65%)
  adding: 118_000.bin (deflated 64%)
  adding: 119_000.bin (deflated 64%)
  adding: 120_000.bin (deflated 64%)
  adding: 12_000.bin (deflated 67%)
  adding: 121_000.bin (deflated 65%)
  adding

In [ ]:
#create tf-idf denominator for each doc
import math
tfidf_dominator ={}
for term, dfi in inverted.df.items():
  post_list  = posting_per_term[term]
  for tmp in post_list:
    doc_id , f = tmp
    Wij = f/inverted.DL[doc_id]
    tfidf_dominator[doc_id] = tfidf_dominator.get(doc_id,0) + math.pow(Wij*math.log((inverted.N/dfi),2),2)
for doc_id,dom in tfidf_dominator.items():
  tfidf_dominator[doc_id] =math.sqrt(tfidf_dominator.get(doc_id,0))

inverted.tfidf_dom = tfidf_dominator
inverted.write_index('.', 'index')

{12: 0.005219935865776231, 303: 0.005829233804397602, 307: 0.01628895380843329, 308: 0.010837864141918807, 336: 0.0059249630314877995, 339: 0.006050257020742621, 358: 0.004535038820920268, 359: 0.00628441406684601, 569: 0.005826161283791953, 594: 0.005275668358434834, 600: 0.004825772529001091, 620: 0.011665211238773095, 624: 0.005472413981287755, 627: 0.009452448772208294, 643: 0.040316681898873614, 663: 0.013474194609217843, 665: 0.009330757809194913, 676: 0.009146398950611463, 677: 0.006281054935665854, 689: 0.00949953939360577, 690: 0.008345819339163937, 691: 0.009912951661033476, 700: 0.004919420428664279, 701: 0.006105688438167179, 705: 0.012354074460051254, 706: 0.010501372185407027, 717: 0.007046039552012848, 736: 0.005383624954425465, 737: 0.010096290565868655, 738: 0.004716121832132117, 740: 0.011944956008579507, 746: 0.004707190102333758, 751: 0.007244310162352924, 752: 0.01052335969460785, 771: 0.007046793799903313, 783: 0.010572737200619605, 802: 0.0066870466145760995, 803

stemmer

READ POSTING LIST FROM INDEX

df idf

In [ ]:
pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  vr = pages.map(lambda x : (x[0],0))
  edges2 = (pages.flatMap(lambda x : [(x[0],p[0]) for p in x[1] ]))
  edges3 = edges2.distinct()


  vr_pre = edges3.flatMap(lambda x: x).distinct()

  edges = edges3.map(lambda x : Row(x[0],x[1]))
  vertices = vr_pre.map(lambda x : Row(x))

  return edges, vertices

In [ ]:
edges, vertices = generate_graph(pages_links)
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")

Page Rank